In [ ]:
import sys
# Install required packages
!{sys.executable} -mpip -q install matplotlib seaborn statsmodels pandas publicdata metapack

%matplotlib inline

import pandas as pd
import geopandas as gpd
import numpy as np
import metapack as mp
import rowgenerators as rg
import publicdata as pub
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

from sklearn import  linear_model
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.stats import weibull_min, lognorm, logistic, norm
from scipy.optimize import curve_fit
from scipy.special import erf

In [ ]:
from scipy.integrate import odeint

def run_sir(t, N, beta, gamma):
    
    #N = 1000
    S = N - 1
    I = 1
    R = 0
    #beta = .6 # infection rate
    #gamma = 0.2 # recovery rate

    R0 = beta / gamma

    # differential equatinons
    def diff(sir, t):
        # sir[0] - S, sir[1] - I, sir[2] - R
        dsdt = - (beta * sir[0] * sir[1])/N
        didt = (beta * sir[0] * sir[1])/N - gamma * sir[1]
        drdt = gamma * sir[1]
        #print (dsdt + didt + drdt)
        dsirdt = [dsdt, didt, drdt]
        return dsirdt

    # initial conditions
    sir0 = (S, I, R)

    # solve ODE
    # the parameters are, the equations, initial conditions, 
    # and time steps (between 0 and 100)
    sir = odeint(diff, sir0, t)

    return pd.Series( N - sir[:, 0], index=t)

def sir_fit_func(t, N, beta, gamma, a, b):
        
    try:
        t = t.round()
        s = run_sir(t, N, beta, gamma)
        return (a*s).shift(int(np.round(b)))
            
    except AttributeError: # It is a scalar, not an array
        print(type(t))
        raise
        t_ = np.linspace(0, 99, num=100)
        s = run_sir(t_, N, beta, gamma)

N = 1000
max_deaths =  11 # t.deaths_mean.cumsum().max() / N
        
sir_fit_func.popt = (N, .34, .19, 70, 30)


popt, pcov = curve_fit(sir_fit_func, cat.day, cat.positive, p0=sir_fit_func.popt)
#popt = sir_fit_func.popt

print(popt)

#cdf['norm'] = func_norm(x, *popt)


fig, ax = plt.subplots(figsize=(12,8))
#ax.plot(t.day, t.allbed_mean.cumsum())
sns.scatterplot(ca.day, ca.positive, ax=ax)
plt.plot(t.day, sir_fit_func(t.day, *popt))

